In [97]:
import yfinance as yf
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from IPython.display import clear_output

In [43]:
alphabet_list=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P',
              'Q','R','S','T','U','V','W','X','Y','Z']
headers = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36"}
fdf = pd.DataFrame(columns=['Short_Ticker','Name','Sector'])
for letter in alphabet_list:
    for page_num in range(1,11):
        try:
            response = requests.get(
                f'https://etfdb.com/alpha/{letter}/#etfs&sort_name=symbol&sort_order=asc&page={page_num}',headers=headers)
        except Exception:
            continue
        soup = BeautifulSoup(response.content.decode(),features='lxml')
        data = []
        table = soup.find('table', attrs={'id':'etfs'})
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele]) # Get rid of empty values
        df = pd.DataFrame(data)   
        df = df.rename(columns={0: "Short_Ticker", 1: "Name",2:"Sector"})
        fdf = pd.concat([fdf, df[['Short_Ticker','Name','Sector']]], ignore_index=True)
fdf

,Short_Ticker,Name,Sector
0,AAAU,Perth Mint Physical Gold ETF,Precious Metals
1,AADR,AdvisorShares Dorsey Wright ADR ETF,Global Equities
2,AAXJ,iShares MSCI All Country Asia ex-Japan ETF,Asia Pacific Equities
3,ABEQ,Absolute Core Strategy ETF,All Cap Equities
4,ACES,ALPS Clean Energy ETF,Alternative Energy Equities
...,...,...,...
6185,ZHOK,SPDR Solactive Hong Kong ETF,Asia Pacific Equities
6186,ZIG,Acquirers Fund,Large Cap Blend Equities
6187,ZJPN,SPDR Solactive Japan ETF,Japan Equities
6188,ZROZ,PIMCO 25+ Year Zero Coupon US Treasury Index Fund,Government Bonds


In [44]:
fdf.to_csv('ETF_List.csv')

In [80]:
# alphabet_list=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P',
#               'Q','R','S','T','U','V','W','X','Y','Z']
# headers = {
#     'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36"}
# fdf2 = pd.DataFrame(columns=['Short_Ticker','Name'])
# for letter in alphabet_list:
#         response = requests.get(
#             f'https://eoddata.com/stocklist/NYSE/{letter}.htm',headers=headers)
#         soup = BeautifulSoup(response.content.decode(),features='lxml')
#         data2 = []
#         table = soup.find('table', attrs={'class':'quotes'})
#         rows = table.find_all('tr')
#         for row in rows[1:]:
#             cols = row.find_all('td')
#             cols = [ele.text.strip() for ele in cols]
#             data2.append([ele for ele in cols if ele]) # Get rid of empty values
#         df2 = pd.DataFrame(data2)   
#         df2 = df2.rename(columns={0: "Short_Ticker", 1: "Name"})
#         fdf2 = pd.concat([fdf2, df2[['Short_Ticker','Name']]], ignore_index=True)
sectors=[]
for ticker in list(fdf2['Short_Ticker']):
    response = requests.get(f'https://finance.yahoo.com/quote/{ticker}/profile?p={ticker}',headers=headers)
    soup = BeautifulSoup(response.content.decode(),features='lxml')
    try:
        sector = soup.find('span', attrs={'class':'Fw(600)'})
        sectors.append(sector.text)
    except Exception:
        sectors.append(np.nan)
fdf2['Sector'] = sectors
fdf2

,Short_Ticker,Name,Sector
0,A,Agilent Technologies,Healthcare
1,AA,Alcoa Corp,Basic Materials
2,AAC,Aac Holdings Inc,NaN
3,AAC.U,Ares Acquisition Corporation Units,Asda store workers win equal pay fight in Supr...
4,AAI-B,Arlington Asset Investment Corp,Asda store workers win equal pay fight in Supr...
...,...,...,...
3478,ZTO,Zto Express Cayman Inc,Industrials
3479,ZTR,Virtus Global Divd & Income Fund,Financial
3480,ZTS,Zoetis Inc Cl A,Healthcare
3481,ZUO,Zuora Inc,Technology


In [81]:
fdf2.to_csv('fdf2.csv')

In [90]:
nasdaq = pd.read_csv('nasdaq.csv')
nasdaq=nasdaq.rename(columns={'Symbol': "Short_Ticker"})

In [91]:
tickers_df = pd.concat([fdf,fdf2,nasdaq]).drop_duplicates().reset_index(drop=True)

In [92]:
tickers_df

,Short_Ticker,Name,Sector
0,AAAU,Perth Mint Physical Gold ETF,Precious Metals
1,AADR,AdvisorShares Dorsey Wright ADR ETF,Global Equities
2,AAXJ,iShares MSCI All Country Asia ex-Japan ETF,Asia Pacific Equities
3,ABEQ,Absolute Core Strategy ETF,All Cap Equities
4,ACES,ALPS Clean Energy ETF,Alternative Energy Equities
...,...,...,...
11525,ZWRKU,Z-Work Acquisition Corp. Units,Finance
11526,ZWRKW,Z-Work Acquisition Corp. Warrant,Finance
11527,ZYME,Zymeworks Inc. Common Shares,Health Care
11528,ZYNE,Zynerba Pharmaceuticals Inc. Common Stock,Health Care


In [99]:
tickers_data = []
for ticker in tqdm(list(tickers_df['Short_Ticker']), position = 0, leave = True):
    try:
        data = yf.download(ticker, start='2005-01-01', end='2021-01-05',interval="1d",progress=False)
    except:
        continue
    clear_output(wait=True)
    data.reset_index(inplace=True)
    data['Date']=data['Date'].astype(str)
    data['Daily_Change'] = pd.DataFrame(data['Adj Close']).pct_change(periods=1, limit=None, freq=None)
    for index, row in data.iterrows():
        tickers_data.append([
                ticker,row['Date'][:10],row['Adj Close'],row['Daily_Change'],row['Volume']])
df1=pd.DataFrame(tickers_data, columns=[
    'Short_Ticker','Date','Adj Close','Daily Change','Volume'])  
df1.head()

100%|██████████████████████████████████████████████████████████████████████████| 11530/11530 [6:13:07<00:00,  1.94s/it]


,Short_Ticker,Date,Adj Close,Daily Change,Volume
0,AAAU,2018-08-15,11.74,NaN,27300.0
1,AAAU,2018-08-16,11.74,0.000000,428400.0
2,AAAU,2018-08-17,11.82,0.006814,52400.0
3,AAAU,2018-08-20,11.90,0.006768,28700.0
4,AAAU,2018-08-21,11.93,0.002521,30600.0


In [101]:
df1.to_csv('tickers_data.csv')

In [105]:
df2 = df1.sort_values(by='Date', ascending=True)
df2.reset_index(inplace=True,drop=True)
df2.head()

,Short_Ticker,Date,Adj Close,Daily Change,Volume
0,ESS,2004-12-31,48.540249,NaN,206400.0
1,SSD,2004-12-31,27.593338,NaN,115800.0
2,HPF,2004-12-31,6.277964,NaN,50800.0
3,INOD,2004-12-31,9.840000,NaN,294500.0
4,MT,2004-12-31,88.868927,NaN,47500.0


In [106]:
df2.to_csv('tickers_data.csv')

In [107]:
tickers_dict = {}
for index, row in tickers_df.iterrows():
    tickers_dict[row['Short_Ticker']]=[row['Name'],row['Sector']]

In [110]:
#df2['Name'] = [tickers_dict[x][0] for x in df2['Short_Ticker']]
df2['Sector'] = [tickers_dict[x][1] for x in df2['Short_Ticker']]
df2.head()

,Short_Ticker,Date,Adj Close,Daily Change,Volume,Name,Sector
0,ESS,2004-12-31,48.540249,NaN,206400.0,Essex Property Trust Inc. Common Stock,Consumer Services
1,SSD,2004-12-31,27.593338,NaN,115800.0,Simpson Manufacturing Company Inc. Common Stock,Capital Goods
2,HPF,2004-12-31,6.277964,NaN,50800.0,John Hancock Pfd Income Fund II Pfd Income Fun...,NaN
3,INOD,2004-12-31,9.840000,NaN,294500.0,Innodata Inc. Common Stock,Technology
4,MT,2004-12-31,88.868927,NaN,47500.0,Arcelor Mittal NY Registry Shares NEW,Basic Industries


In [111]:
len(df2.Short_Ticker.unique())

6722

In [112]:
df3 = df2[['Short_Ticker','Name','Sector','Date', 'Adj Close', 'Daily Change', 'Volume']]
df3.head()

,Short_Ticker,Name,Sector,Date,Adj Close,Daily Change,Volume
0,ESS,Essex Property Trust Inc. Common Stock,Consumer Services,2004-12-31,48.540249,NaN,206400.0
1,SSD,Simpson Manufacturing Company Inc. Common Stock,Capital Goods,2004-12-31,27.593338,NaN,115800.0
2,HPF,John Hancock Pfd Income Fund II Pfd Income Fun...,NaN,2004-12-31,6.277964,NaN,50800.0
3,INOD,Innodata Inc. Common Stock,Technology,2004-12-31,9.840000,NaN,294500.0
4,MT,Arcelor Mittal NY Registry Shares NEW,Basic Industries,2004-12-31,88.868927,NaN,47500.0


In [113]:
df3.to_csv('tickers_data.csv')